In [29]:
import json
import os
import shutil
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model
from keras.preprocessing.image import img_to_array
import cv2

In [86]:
f = open('labels/train.json')
train = json.load(f)

## Select Images Including Traffic Lights

In [101]:
# select images including traffic_lights
traffic_lights = {}
for item in train:
    name = item['name']
    
    for label in item['labels']:
        if label['category'] == 'traffic light':
            traffic_lights[name] = []
            break
    for label in item['labels']:        
        if label['category'] == 'traffic light':
            bbox = label['box2d']
            traffic_lights[name].append(bbox)
            

# copy them to another folder
for img in traffic_lights:
    shutil.copyfile('bdd100k/'+img, 'traffic_lights/'+img)

## Day-Night Classification

In [73]:
model = load_model('day_night.hd5')

In [102]:
# It takes a while!
# 0 = night , 1 = day
for img in os.listdir('traffic_lights'):
    
    image = cv2.imread('traffic_lights/'+img)
    image = cv2.resize(image, (28, 28))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    label = model.predict(image)
    
    if label[0][0] > label[0][1]:
        traffic_lights[img].append(0)
    else:
        traffic_lights[img].append(1)

In [108]:
tl = traffic_lights

# Remove night images from dic
for key, value in tl.copy().items():
    if value[-1] == 0:
        del tl[key]
        
# Copy day images to another folder        
for key in tl:
    shutil.copyfile('traffic_lights/'+key, 'traffic_lights_day/'+key)

## Convert Annotation

In [126]:
# remove label
for value in tl.values():
    del value[-1]
    
# save annotations
with open('day_traffic_lights.json', 'w') as fp:
    json.dump(tl, fp)

In [25]:
f = open('day_traffic_lights.json')
train = json.load(f)

In [26]:
# [xmin, ymin, xmax, ymax] > [xcenter, ycenter, objwidth, objheight]
for name, value in train.items():
    # check image sizes
    im = cv2.imread('traffic_lights_day/'+name)
    h, w, c = im.shape
    
    for i, obj in enumerate(value):
        value[i] = [ 1, (obj['x2'] + obj['x1']) / (2*w), (obj['y2'] + obj['y1']) / (2*h), (obj['x2'] - obj['x1']) / w, (obj['y2'] - obj['y1']) / h]

In [32]:
# save them
fmt = '%.10g'

for name, obj in train.items():
    np.savetxt('labels/'+name.replace(name[-4:],'.txt'), obj, newline='\n',fmt=fmt)